In [1]:
import polars as pl

In [3]:
awards = pl.read_csv('./lahman_1871-2023_csv/AwardsPlayers.csv')
awards.head()

playerID,awardID,yearID,lgID,tie,notes
str,str,i64,str,str,str
"""poseybu01""","""Lou Gehrig Memorial Award""",2019,"""ML""",null,null
"""klubeco01""","""Lou Gehrig Memorial Award""",2018,"""ML""",null,null
"""vottojo01""","""Lou Gehrig Memorial Award""",2017,"""ML""",null,null
"""altuvjo01""","""Lou Gehrig Memorial Award""",2016,"""ML""",null,null
"""grandcu01""","""Lou Gehrig Memorial Award""",2015,"""ML""",null,null


In [2]:
fielders = pl.read_csv('./lahman_1871-2023_csv/Fielding.csv')
fielders.head()


playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str
"""aardsda01""",2004,1,"""SFN""","""NL""","""P""",11,0,32,0,0,0,0,null,null,null,null,null
"""aardsda01""",2006,1,"""CHN""","""NL""","""P""",45,0,159,1,5,0,1,null,null,null,null,null
"""aardsda01""",2007,1,"""CHA""","""AL""","""P""",25,0,97,2,4,1,0,null,null,null,null,null
"""aardsda01""",2008,1,"""BOS""","""AL""","""P""",47,0,146,3,6,0,0,null,null,null,null,null
"""aardsda01""",2009,1,"""SEA""","""AL""","""P""",73,0,214,2,5,0,1,null,null,null,null,null


In [10]:
(fielders
 .group_by(pl.col('playerID'),pl.col('yearID'))
.agg(pl.col('stint').count().alias('count_of_positions/teams'))
.sort(pl.col('count_of_positions/teams')))

playerID,yearID,count_of_positions/teams
str,i64,u32
"""galvipu01""",1889,1
"""faheybi01""",1971,1
"""huskebu01""",1998,1
"""gibauia01""",2020,1
"""baldwbi01""",1976,1
…,…,…
"""seitzke01""",1993,10
"""kellyki01""",1891,10
"""bellda01""",1998,10


In [5]:
winners = (awards
 .filter(pl.col('notes') != 'null',pl.col('awardID') =='Gold Glove'))

In [30]:
print(fielders.columns)

['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR']


In [21]:
(fielders
 .filter(pl.col('yearID') >= 2013)
 .group_by(pl.col('playerID','yearID','lgID','POS'))
 .agg(pl.col(['G', 'GS', 'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR']).sum())
# .agg(pl.col('G').sum(),pl.col('GS').sum(),pl.col('InnOuts').sum(),pl.col('PO').sum(),pl.col('A').sum(),pl.col('E').sum())
 .join(winners,on = ['playerID','yearID'],how='left')
.with_columns(Gold_Glove = pl.when(pl.col('awardID') =='Gold Glove')
                                .then(pl.lit('Yes'))
                                .otherwise(pl.lit('No')))
.sort(pl.col('playerID'))
)

playerID,yearID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,awardID,lgID_right,tie,notes,Gold_Glove
str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str
"""aardsda01""",2015,"""NL""","""P""",33,0,92,0,1,1,0,null,null,null,null,null,null,null,null,null,"""No"""
"""aardsda01""",2013,"""NL""","""P""",43,0,119,1,5,0,0,null,null,null,null,null,null,null,null,null,"""No"""
"""abadfe01""",2023,"""NL""","""P""",6,0,19,0,0,1,0,null,null,null,null,null,null,null,null,null,"""No"""
"""abadfe01""",2019,"""NL""","""P""",21,0,39,0,0,0,0,null,null,null,null,null,null,null,null,null,"""No"""
"""abadfe01""",2021,"""AL""","""P""",16,0,53,0,3,0,0,null,null,null,null,null,null,null,null,null,"""No"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zuninmi01""",2014,"""AL""","""C""",130,125,3363,1010,84,5,5,null,null,null,null,null,null,null,null,null,"""No"""
"""zuninmi01""",2016,"""AL""","""C""",52,48,1331,400,15,0,0,null,null,null,null,null,null,null,null,null,"""No"""
"""zychto01""",2017,"""AL""","""P""",45,0,122,1,3,0,0,null,null,null,null,null,null,null,null,null,"""No"""
